In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import lognorm
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.metrics import mean_absolute_error

In [ ]:
in_file_name = 'data/grouped_paces_ju.tsv'
df_all = pd.read_csv(in_file_name, delimiter="\t")

In [ ]:
#df = df_all[np.isfinite(df_all.pace_1)]
df = df_all

In [ ]:
paces = df.as_matrix(["pace_2", "pace_3", "pace_4", "pace_5", "pace_6"])
logs = np.log(paces)
means = np.nanmean(logs, axis=1)
stdevs = np.nanstd(logs, axis=1)
df = df.assign(log_means=pd.Series(means).values)
df = df.assign(log_stdevs=pd.Series(stdevs).values)

In [ ]:
# Estimate values for all, but only use them if not other value is available
with_history = df[np.isfinite(df_all.pace_3)]
x = with_history.as_matrix(["mean_team_id"])
x = x.reshape(len(x), 1)

log_means = with_history.log_means.values.reshape(len(with_history.log_means), 1)
log_means_model = linear_model.LinearRegression()
log_means_model.fit(x, log_means)
estimated_log_means = log_means_model.predict(df.mean_team_id.values.reshape(len(df.mean_team_id), 1))
df = df.assign(estimated_log_means=estimated_log_means)

log_stdevs = with_history.log_stdevs.values.reshape(len(with_history.log_stdevs), 1)
log_stdevs_model = linear_model.LinearRegression()
log_stdevs_model.fit(x, log_stdevs)
estimated_log_stdevs = log_stdevs_model.predict(df.mean_team_id.values.reshape(len(df.mean_team_id), 1))
df = df.assign(estimated_log_stdevs=estimated_log_stdevs)



In [ ]:
df

In [ ]:
# s = sigma and scale = exp(mu).
df['final_log_means'] = np.where(np.isfinite(df["log_means"]) & df["log_means"]>0, df["log_means"], df["estimated_log_means"])
final_log_means = np.exp(df['final_log_means']) 
df['final_log_stdevs'] = np.where(np.isfinite(df["log_stdevs"]) & df["log_stdevs"]>0, df["log_stdevs"], df["estimated_log_stdevs"])
final_log_stdevs = df['final_log_stdevs']

intervals95 = lognorm.interval(0.95, s = final_log_stdevs, scale = final_log_means)
means = lognorm.mean(s = final_log_stdevs, scale = final_log_means)
medians = lognorm.median(s = final_log_stdevs, scale = final_log_means)

In [ ]:

df = df.assign(interval95_start = pd.Series( intervals95[0] ).values)
df = df.assign(interval95_end = pd.Series( intervals95[1] ).values)
df = df.assign(p1_in_interval = pd.Series( (df.interval95_start <= df.pace_1) & (df.interval95_end >= df.pace_1) ).values)
df = df.assign(mean_ln = pd.Series(means).values)
df = df.assign(med_ln = pd.Series(medians).values)

In [ ]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())
mean_err = rmse(df.mean_ln, df.pace_1)
med_err = rmse(df.med_ln, df.pace_1)
(mean_err, med_err)

In [ ]:
np.mean(df.p1_in_interval)

In [ ]:
df.to_csv('data/log_normal_estimates_ju.tsv', sep="\t")

In [ ]:
runs17 = pd.read_csv('data/csv-results_j2017_ju.tsv', delimiter="\t")


In [ ]:
def get_estimate_row(row):
    name = row["competitor-name"].lower()
    
    by_name = df[df['name'] == name]
    by_name_and_colon = df[df['name'].str.contains(name +":", regex=False)]

    runners = by_name.append(by_name_and_colon)
    if(len(runners) == 1):
        return runners
    team_name = row["team-name"].upper()
    runners = runners[runners['teams'].str.contains(team_name, regex=False)]
    if(len(runners) == 1):
        return runners
    print(f"name '{name}' team_name '{team_name}'")
    print(f"by_name {len(by_name)} by_name_and_colon {len(by_name_and_colon)} runners {len(runners)}")
    print(f"Duplicate runner {runners}")
    #print(f"TEAMS by_name_and_colon {by_name_and_colon['teams']}")
    return runners.sort_values("num_runs", ascending = False).head(1)

def get_estimate_params(row):
    estimate_row = get_estimate_row(row)
    #print(f"estimate_row final_log_means {estimate_row.final_log_means} {estimate_row.final_log_stdevs}")
    final_log_means = estimate_row.final_log_means.values[0]
    final_log_stdevs = estimate_row.final_log_stdevs.values[0]
    return pd.Series({"final_log_means": final_log_means, "final_log_stdevs": final_log_stdevs})

#runs17 = runs17[runs17['team-name'].str.contains("Reak")]
estimate_params = runs17.apply(lambda row: get_estimate_params(row), axis=1)
runs17 = runs17.assign(final_log_means = estimate_params.final_log_means)
runs17 = runs17.assign(final_log_stdevs = estimate_params.final_log_stdevs)

In [ ]:
def leg_dist(leg):
    dist = [12.7, 14.2, 12.3, 7.6, 7.9, 10.9, 13.8]
    return dist[leg - 1]

runs17 = runs17.assign(leg_dist = runs17["leg-nro"].apply(lambda nro: leg_dist(nro)))

final_means = np.exp(runs17.final_log_means)
#intervals95 = lognorm.interval(0.95, s = runs17.final_log_stdevs, scale = final_means)

#runs17 = runs17.assign(start95 = intervals95[0] * runs17["leg_dist"])
#runs17 = runs17.assign(end95 = intervals95[1] * runs17["leg_dist"])

runs17["est_median"] = lognorm.median(s = runs17.final_log_stdevs, scale = final_means) * runs17["leg_dist"]


In [ ]:
runs17

In [ ]:
by_teams = runs17.set_index(["team-id", "leg-nro"]).unstack()
by_teams.head()

In [ ]:
for i in range(1,8):
    by_teams["fin_real", i] = np.sum([ by_teams["leg-time"][j] / 60 for j in range(1,i+1) ], axis=0)
by_teams.fin_real.head()

In [ ]:
for i in range(1,8):
    by_teams["fin_med", i] = np.sum([ by_teams["est_median"][j] for j in range(1,i+1) ], axis=0)
by_teams.fin_med.head()

In [ ]:

def get_simulated_medians(row):
    samples = pd.DataFrame()
    for i in range(1,8):
        samples[i] = row["leg_dist"][i] * lognorm.rvs(s = row["final_log_stdevs"][i], scale = np.exp(row["final_log_means"][i]), size = 10000)

    samples_sums = pd.DataFrame()
    for i in range(1,8):
        samples_sums[i] = np.sum([ samples[j] for j in range(1,i+1) ], axis=0)

    start95 = samples_sums.quantile(0.025)
    end95 = samples_sums.quantile(0.975)
    medians = samples_sums.median()
    means = samples_sums.mean()
    
    """
    for i in range(1,8):
        bins = int(samples_sums[i].max() - samples_sums[i].min())
        name = row["competitor-name"][i]
        plt.title(f"{name} bins = {bins}")
        plt.hist(samples_sums[i], bins=bins)
        plt.axvline(x=row["fin_real"][i], color="r")        
        plt.axvline(x=medians[i], color="g")
        plt.axvline(x=means[i], color="yellow")
        plt.show()
    """
    
    """
    bins = int( (samples_sums.max().max() - samples_sums.min().min()) / 5) 
    plt.figure(figsize=(20, 6))
    plt.title(f"Whole team, bins = {bins}")
    plt.hist([samples_sums[1], samples_sums[2], samples_sums[3], samples_sums[4], samples_sums[5], samples_sums[6], samples_sums[7]], bins=bins)
    for i in range(1,8):
        plt.axvline(x=row["fin_real"][i], color="r")
        plt.axvline(x=medians[i], color="g")

    plt.show()
    """
    
    fin_start95_dict = {f"fin_start95_{leg}" : start95.values[leg-1] for leg in range(1,8)}
    fin_end95_dict = {f"fin_end95_{leg}" : end95.values[leg-1] for leg in range(1,8)}
    fin_median_dict = {f"fin_median_{leg}" : medians.values[leg-1] for leg in range(1,8)}
    fin_mean_dict = {f"fin_mean_{leg}" : means.values[leg-1] for leg in range(1,8)}
    new_cols = {**fin_start95_dict, **fin_end95_dict, **fin_median_dict, **fin_mean_dict}

    #print(start95.values)
    #print(new_cols)
    return pd.Series(new_cols)

simulated = by_teams.apply(get_simulated_medians, axis=1)


simulated.head()

In [ ]:
by_teams_2 = by_teams.copy()
by_teams_2.columns = [f'{x[0]}_{x[1]}' for x in by_teams_2.columns]
by_teams_2.reset_index()
by_teams_2.head()

In [ ]:
result = pd.concat([by_teams_2, simulated], axis=1, join='inner')
result.head()

In [ ]:
start_timestamp = pd.Timestamp(year = 2017, month = 6, day = 17, hour = 23)
result["fint_real_1"] = pd.to_datetime(result["fin_real_1"] * 60, unit = "s", origin= start_timestamp)

for leg in range(1,8):
    result[f"fint_real_{leg}"] = pd.to_datetime(result[f"fin_real_{leg}"] * 60, unit = "s", origin= start_timestamp)
    result[f"fint_median_{leg}"] = pd.to_datetime(result[f"fin_median_{leg}"] * 60, unit = "s", origin= start_timestamp)
    result[f"fint_start95_{leg}"] = pd.to_datetime(result[f"fin_start95_{leg}"] * 60, unit = "s", origin= start_timestamp)
    result[f"fint_end95_{leg}"] = pd.to_datetime(result[f"fin_end95_{leg}"] * 60, unit = "s", origin= start_timestamp)

result["fint_end95_2"]  
    

In [ ]:
result.to_csv('data/team_estimates_ju2017.tsv', sep="\t")

In [ ]:
[ (rmse(result[f"fin_mean_{leg}"], result[f"fin_real_{leg}"]), rmse(result[f"fin_median_{leg}"], result[f"fin_real_{leg}"])) for leg in range(1,8)]

In [ ]:
result["fin_real_7"].head()

In [ ]:
qualified = result[np.isfinite(result.fin_real_7)]

In [ ]:
[ (rmse(qualified[f"fin_mean_{leg}"], qualified[f"fin_real_{leg}"]), rmse(qualified[f"fin_median_{leg}"], qualified[f"fin_real_{leg}"])) for leg in range(1,8)]

In [ ]:
[ (mean_absolute_error(qualified[f"fin_real_{leg}"], qualified[f"fin_mean_{leg}"]), mean_absolute_error(qualified[f"fin_real_{leg}"], qualified[f"fin_median_{leg}"])) for leg in range(1,8)]

In [ ]:
np.mean((result["fin_start95_1"] < result["fin_real_1"]) & (result["fin_end95_1"] > result["fin_real_1"]) )

In [ ]:
np.mean((qualified["fin_start95_1"] < qualified["fin_real_1"]) & (qualified["fin_end95_1"] > qualified["fin_real_1"]) )

### 